# 2장: 메지시 전달(Message Passing)
## 2.1 빌트인 함수 및 메지시 전달 API들

In [ ]:
def message_func(edges):
     return {'he': edges.src['hu'] + edges.dst['hv']}

In [ ]:
import torch
def reduce_func(nodes):
     return {'h': torch.sum(nodes.mailbox['m'], dim=1)}

In [ ]:
import dgl.function as fn
graph.apply_edges(fn.u_add_v('el', 'er', 'e'))

In [ ]:
def update_all_example(graph):
    # store the result in graph.ndata['ft']
    graph.update_all(fn.u_mul_e('ft', 'a', 'm'),
                     fn.sum('m', 'ft'))
    # Call update function outside of update_all
    final_ft = graph.ndata['ft'] * 2
    return final_ft

## 2.2 효율적인 메시지 전달 코드 작성 방법¶

In [ ]:
import torch
import torch.nn as nn

linear = nn.Parameter(torch.FloatTensor(size=(node_feat_dim * 2, out_dim)))
def concat_message_function(edges):
     return {'cat_feat': torch.cat([edges.src['feat'], edges.dst['feat']], dim=1)}
g.apply_edges(concat_message_function)
g.edata['out'] = g.edata['cat_feat'] @ linear

In [ ]:
import dgl.function as fn

linear_src = nn.Parameter(torch.FloatTensor(size=(node_feat_dim, out_dim)))
linear_dst = nn.Parameter(torch.FloatTensor(size=(node_feat_dim, out_dim)))
out_src = g.ndata['feat'] @ linear_src
out_dst = g.ndata['feat'] @ linear_dst
g.srcdata.update({'out_src': out_src})
g.dstdata.update({'out_dst': out_dst})
g.apply_edges(fn.u_add_v('out_src', 'out_dst', 'out'))

## 2.3 그래프 일부에 메지시 전달 적용하기 : 미니-배치 학습에서 흔히 사용되는 방법

In [ ]:
nid = [0, 2, 3, 6, 7, 9]
sg = g.subgraph(nid)
sg.update_all(message_func, reduce_func, apply_node_func)

## 2.4 메시지 전달에 에지 가중치 적용하기
GAT 또는 일부 GCN 변형 에서 사용되는 것처럼 메시지 병합이전에 에지의 가중치를 적용하는 것은 GNN 모델링에서 흔하게 사용되는 기법이다. DGL은 이를 다음과 같은 밥벙으로 지원하고 있다.
- 가중치를 에지 피쳐로 저장
- 메시지 함수에서 에지 피쳐를 소스 노드의 피쳐와 곱하기

In [ ]:
import dgl.function as fn

# Suppose eweight is a tensor of shape (E, *), where E is the number of edges.
graph.edata['a'] = eweight
graph.update_all(fn.u_mul_e('ft', 'a', 'm'),
                 fn.sum('m', 'ft'))

## 2.5 이종 그래프에서의 메시지 전달
이종 그래프에서 메시지 전달은 두 파트로 나뉜다:
1. 각 관계(relation) r에 대한, 메지시 연산과 집계(aggregation)
2. 가 노트 타입에 대한 모든 관계의 집계 결과를 합치는 축약(reduction)

In [ ]:
import dgl.function as fn

for c_etype in G.canonical_etypes:
    srctype, etype, dsttype = c_etype
    Wh = self.weight[etype](feat_dict[srctype])
    # Save it in graph for message passing
    G.nodes[srctype].data['Wh_%s' % etype] = Wh
    # Specify per-relation message passing functions: (message_func, reduce_func).
    # Note that the results are saved to the same destination feature 'h', which
    # hints the type wise reducer for aggregation.
    funcs[etype] = (fn.copy_u('Wh_%s' % etype, 'm'), fn.mean('m', 'h'))
# Trigger message passing of multiple types.
G.multi_update_all(funcs, 'sum')
# return the updated node feature dictionary
return {ntype : G.nodes[ntype].data['h'] for ntype in G.ntypes}